In [39]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
MIN_PARAGRAPH_LEN = 5

In [13]:
def load_data(_dir):
    ret = []
    for each in os.listdir(_dir):
        full_path = os.path.join(_dir, each)
        if each.endswith("txt"):
            with open(full_path, "r") as f:
                aux = f.read().split('\n\n')
                for paragraph in aux:
                    paragraph = paragraph.strip('\n')
                    paragraph += '\n'
                    if len(paragraph) < MIN_PARAGRAPH_LEN:
                        continue
                    ret.append(paragraph)
    return ret

In [14]:
ps = load_data("./speeches/")

In [15]:
print("# Speeches: {}".format(len(ps)))

arr = np.asarray([len(x) for x in ps])

print("Mean {}".format(np.mean(arr)))
print("Median {}".format(np.median(arr)))
print("Std {}".format(np.std(arr)))
print("Max {}".format(np.max(arr)))
print("Min {}".format(np.min(arr)))

# Speeches: 1246
Mean 643.975922953451
Median 539.0
Std 420.62152407478965
Max 2950
Min 16


In [16]:
for i in range(5):
    print(ps[i])
    print("----------------------------------")

Muchas gracias. ¡Viva, viva la Patria, sí!

----------------------------------
Señores Jefes de Estado presentes; señores Jefes de Delegaciones; señores gobernadores; autoridades civiles, militares, eclesiásticas; pueblo de la Patria y Honorable Asamblea Legislativa: vengo esta tarde a dar cumplimiento al artículo 93 de la Constitución Nacional. Luego de haberse realizado elecciones el 28 de octubre, la fórmula que encabecé junto al ingeniero Julio Cobos, obtuvo más del 45 por ciento de los votos válidos emitidos y, por lo tanto, no corresponde, tal cual ha proclamado esta misma Asamblea Legislativa convocar a una segunda vuelta. En esos términos y en los términos del artículo 97, vengo a tomar posesión del cargo de Presidenta de la República Argentina, el honor más grande que puede tener una argentino o una argentina, ser elegida por sus compatriotas para representarlos.

----------------------------------
Este es un escenario diferente al de hace apenas cuatro años y medio, el 25 de 

In [19]:
def preprocess(paragraphs):
    chars = set()
    
    for each in paragraphs:
        chars.update(set(each))
    
    char_to_ix = { ch:i for i,ch in enumerate(chars) }

    ix_to_char = { i:ch for i,ch in enumerate(chars) }

    max_p = max([len(i) for i in paragraphs])
    
    ret = np.zeros(shape=(len(paragraphs), max_p, len(char_to_ix)))
    lens = np.zeros(shape=len(paragraphs))

    for idx, each in enumerate(paragraphs):
        lens[idx] = len(each)
        for i in range(max_p - len(each)):
            each += '\n'
        aux = np.zeros(shape=(len(each), len(chars)))
        for i, c in enumerate(each):
            aux[i][char_to_ix[c]] = 1
        ret[idx] = aux

        
    return ret, lens, char_to_ix, ix_to_char

In [20]:
data, lens, char_to_ix, ix_to_char = preprocess(ps)

In [21]:
'\n' in char_to_ix

True

In [22]:
print(data.shape)

(1246, 2950, 102)


# Defining model

In [163]:
BATCH_SIZE = 32
INPUT_SIZE = len(ix_to_char)

TIMES = 32
N_HIDDEN = 128

tf.reset_default_graph()


init = tf.contrib.layers.xavier_initializer()
x = tf.placeholder(tf.float32, shape=(TIMES, None, INPUT_SIZE))
y = tf.placeholder(tf.float32, shape=(TIMES, None, INPUT_SIZE))

seq_len = tf.placeholder(tf.int64, shape=(None))

x_2 = tf.unstack(x, axis=0)

init_state_c = tf.placeholder(tf.float32, shape=[None, N_HIDDEN])
init_state_h = tf.placeholder(tf.float32, shape=[None, N_HIDDEN])

cell = tf.contrib.rnn.BasicLSTMCell(N_HIDDEN)
    
t = tf.contrib.rnn.LSTMStateTuple(init_state_c, init_state_h)
outputs, states = tf.contrib.rnn.static_rnn(cell, x_2, dtype=tf.float32, sequence_length=seq_len, initial_state=t)

outputs_2 = tf.stack(outputs, axis=0)

out = tf.layers.dense(outputs_2, units=INPUT_SIZE, kernel_initializer=init)

out_softmax = tf.nn.softmax(out)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=y))

tf.summary.scalar('loss', loss)

merge = tf.summary.merge_all()

upd = tf.train.AdamOptimizer().minimize(loss)

# Running RNN

In [122]:
print(data.shape)

(1246, 2950, 102)


In [ ]:
EPOCHS = 1000

N, M, V = data.shape

sess = tf.Session()
sess.run(tf.global_variables_initializer())

zeros = np.zeros(shape=(BATCH_SIZE))
times_minus_one = (TIMES - 1) * np.ones(shape=(BATCH_SIZE))

train_writer = tf.summary.FileWriter('./logs/train', sess.graph)

counter = 0
for e in tqdm(range(EPOCHS)):
    
    idxs = np.random.choice(N, BATCH_SIZE, replace=False)
    batch = data[idxs]
    batch_lens = lens[idxs].astype(np.int32)
    
    ts = (M-1) // TIMES # + 1
    
    # Initial state
    c = np.zeros((BATCH_SIZE, N_HIDDEN))
    h = np.zeros((BATCH_SIZE, N_HIDDEN))
    
    print("".join(test("a", max_=100)))
    
    for t in range(ts):
        batch_x = batch[:, t*TIMES:TIMES*(t+1), :]
        batch_y = batch[:, t*TIMES+1:TIMES*(t+1)+1, :]
        
        batch_lens_aux = batch_lens -  (TIMES * t)
        
        batch_lens_aux = np.maximum(zeros, batch_lens_aux)
        batch_lens_aux = np.minimum(times_minus_one, batch_lens_aux)
        
        batch_lens_aux = batch_lens_aux.astype(np.uint8)
        
        non_zero_idxs = batch_lens_aux > 0
        batch_lens_aux = batch_lens_aux[non_zero_idxs]

        batch_x = batch_x[:, non_zero_idxs, :]
        batch_y = batch_y[:, non_zero_idxs, :]
        c_l = c[non_zero_idxs]
        h_l = h[non_zero_idxs]
        
        if np.all(batch_lens_aux == 0):
            break
    
           
        m, states_, _ = sess.run([merge, states, upd], feed_dict={x: batch_x, y: batch_y, init_state_c: c_l, init_state_h: h_l, seq_len: batch_lens_aux})
        train_writer.add_summary(m, counter)
        
        counter += 1
        
        c[non_zero_idxs] = states_.c
        h[non_zero_idxs] = states_.h
        


  0%|          | 0/1000 [00:00<?, ?it/s]

ü9S(.lq”NZÍú42g)A9PLuv…ÍYW.”!L`J3k‘c¡IUfx°XBbBE–4ÉS%rT0rs0…kbkLv');ANx;Vv…RWú?czs5É/Á!62Í!2SíW77O¡óTG



  0%|          | 1/1000 [00:06<1:40:03,  6.01s/it]

o3XñfdÚUeG?N



  0%|          | 2/1000 [00:07<1:04:47,  3.90s/it]

...

brio No ha violo sicues cina mbior, a Luel esouduere hajua ¿Quoso co trpl hata savuese entar lo de iv


 10%|█         | 102/1000 [04:11<36:49,  2.46s/it]

.4



 10%|█         | 103/1000 [04:14<36:53,  2.47s/it]

ñoo quegasosumolusore ue ran Plabís ro l o y e ca, 200 quara y do lara qus.0 siolo ciado l púbrto po 


In [224]:
def test(st, max_=1000):
    
    start = st
    pred = start
    
    c = np.zeros((1, N_HIDDEN))
    h = np.zeros((1, N_HIDDEN))
    
    ret = []
        
    while True:
        
        in_ = np.zeros(shape=(TIMES, 1, INPUT_SIZE), dtype=np.uint)
        in_[0, 0, char_to_ix[pred]] = 1

        net_out, net_states = sess.run([out_softmax, states], feed_dict={x: in_, init_state_c: c, init_state_h: h, seq_len: np.ones(shape=(1,))})
        c, h = net_states.c, net_states.h
        char_out = ix_to_char[int(np.random.choice(np.arange(INPUT_SIZE), p=np.squeeze(net_out[0])))]
        
        ret.append(char_out)

        pred = char_out
                                                                         
        if char_out == '\n' or len(ret) > max_:
            break
        
    return ret                                                                       

In [226]:
print("".join(test("b")))

afo e es A tase menste de es liotía y le liten y Impoteetodemone, un halontidocora te uen plo qu? déduntas 8tos Aicade guesar que ue biéntotos n o emmecio, as elostibio horiétos ne Areotrs eí, obado ueconton von doco e te do varciesivadomo mprmba a4.ñosu mprterran sjegade castrar mbila munasice la da lue fu r sriólon, itano sasicuestimánte diéelastameci quemásta lon esiéle C

